## Hold Overnight Strategy Eval

Buy stock at the close and hold until the open. Hypothesis is that traders are unwilling to hold overnight?

Evaluation steps:

1. Pull stocks from Xano
2. Get historical prices on each stock
3. Calculate the change from the previous close

In [87]:
%matplotlib inline

import requests
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc

pd.set_option('display.max_columns', None)

In [2]:
url = 'https://iocalc.com/api:gPr_YnZX/getstocks'

response = requests.get(url)

stock_table = pd.json_normalize(response.json()['stocks'])
stock_table

,id,created_at,name,symbol,price,target,marketcap,week52high,week52low,dividendyield,day200movingavg,day50movingavg,exdividend,nextearnings,eps,pe,beta,change_1yr,change_ytd,pricetargethigh,pricetargetlow,symboltype,analysts,currency,totaltracking
0,19,1670219209825,Global X Funds,QYLD,16.62,0.00,5871377700,22.43,19.11,0.121877,21.72,21.91,2022-01-24,None,,,,-0.0083631439208244,-0.079315006759802,0.0,0.0,,0,,0
1,20,1670219253436,Meta Financial Group Inc.,CASH,43.80,72.50,1741265522,65.9,36.02,0.003454,55.53,59.15,2021-12-08,2022-01-26,8.76,6.2630944593554,1.1439973981583,0.48746215755319,-0.030673818303721,79.0,70.0,stock,4,USD,0
2,11,1670219336776,Verizon Communications Inc,VZ,38.18,60.00,223153658344,58.44,49.09,0.058884,52.74,52.07,2022-01-07,2022-01-25,5.32,10.124939126336,0,-0.032254541085492,0.035421706459115,71.0,52.0,stock,26,USD,0
3,49,1670219547079,"Delta Air Lines, Inc.",DAL,35.71,51.56,24262923840,52.28,33.4,0.000000,40.26,38.95,None,2022-04-14,-0.1064,-356.807703535,1.1989249496749,-0.05225,-0.029938587512794,67.0,40.0,stock,18,USD,0
4,54,1670220552059,Humana Inc.,HUM,546.78,485.87,48727349718,473.14,351.2,0.007368,427.87,430.11,2021-12-30,2022-02-02,20.51,18.229461173999,0.54182997393569,-0.058274921620204,-0.18272754710473,581.0,410.0,stock,23,USD,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,74,1643141926509,"Oceaneering International, Inc.",OII,13.88,17.72,1282413359,18.2,8.27,0.000000,12.87,12.16,None,2022-02-24,-0.3555,-36.130426529836,1.8863947677163,0.21916508538899,0.13616268788683,20.0,16.0,stock,9,USD,0
66,65,1643141629684,ALPS Fund Services,SBIO,33.95,0.00,146072500,64.04,31.96,0.000000,44.26,40.35,2021-12-22,None,,,,-0.38581129378127,-0.18631628787879,0.0,0.0,,0,,0
67,68,1656705855826,Rocket Lab USA Inc,RKLB,3.81,21.20,3917063728,21.34,8,0.000000,13.36,11.44,None,2022-03-30,0,0,0,-0.16395014381592,-0.28990228013029,28.0,18.0,stock,5,USD,0
68,70,1670219260310,Kosmos Energy Ltd,KOS,6.42,5.96,1916009623,4.89,1.8,0.000000,3.15,3.92,None,2022-02-24,-0.4139,-11.364502259128,2.3177199524754,0.58208955223881,0.22543352601156,8.0,4.1,stock,7,USD,0


In [3]:
config = pd.read_csv('../tradier_exploration/sandbox.config')
token = config.prod_token[0]

In [4]:
endpoint = 'https://api.tradier.com'
path = '/v1/markets/history'

all_stock_history = pd.DataFrame()

for symbol in stock_table['symbol']:
    response = requests.get(f'{endpoint}{path}',
        params={'symbol':f'{symbol}', 
                'interval': 'daily', 
                'start': '2000-01-01', 
                'end': f'{datetime.today().date().isoformat()}'},
        headers={'Authorization': f'Bearer {token}', 
                 'Accept': 'application/json'}
    )
    json_response = response.json()
    this_stock = pd.json_normalize(json_response['history']['day'])
    this_stock['symbol'] = symbol
    
    all_stock_history = pd.concat([all_stock_history, this_stock], axis = 0, ignore_index = True)

In [5]:
all_stock_history['yest_close'] = all_stock_history.groupby(['symbol'])['close'].shift(1)
all_stock_history

,date,open,high,low,close,volume,symbol,yest_close
0,2014-01-15,25.75,25.75,25.6394,25.670,6503,QYLD,NaN
1,2014-01-16,25.69,25.7,25.6413,25.700,2910,QYLD,25.670
2,2014-01-17,25.67,25.77,25.65,25.710,5615,QYLD,25.700
3,2014-01-21,25.82,25.74,25.66,25.719,2230,QYLD,25.710
4,2014-01-22,25.529,25.54,25.529,25.530,1055,QYLD,25.719
...,...,...,...,...,...,...,...,...
256441,2022-11-29,64.1,64.25,62.71,62.870,2061764,GFS,63.900
256442,2022-11-30,62.5,64.45,60.45,64.350,2418476,GFS,62.870
256443,2022-12-01,64.32,64.43,61.87,63.660,1617077,GFS,64.350
256444,2022-12-02,61.88,62.69,60.72,62.540,1755186,GFS,63.660


In [7]:
all_stock_history['yest_close'] = all_stock_history.groupby(['symbol'])['close'].shift(1)
all_stock_history['open'] = all_stock_history['open'].astype(np.float64)
all_stock_history['tom_open'] = all_stock_history.groupby(['symbol'])['open'].shift(-1)
all_stock_history['overnight_return'] = all_stock_history['open'] - all_stock_history['yest_close']
all_stock_history['overnight_return_perc'] = ((all_stock_history['open'] - 
                                               all_stock_history['yest_close']) / 
                                              all_stock_history['yest_close'])
all_stock_history['next_night_return'] = all_stock_history['tom_open'] - all_stock_history['close']
all_stock_history

,date,open,high,low,close,volume,symbol,yest_close,overnight_return,overnight_return_perc,tom_open,next_night_return
0,2014-01-15,25.750,25.75,25.6394,25.670,6503,QYLD,NaN,NaN,NaN,25.690,0.02
1,2014-01-16,25.690,25.7,25.6413,25.700,2910,QYLD,25.670,0.02,0.000779,25.670,-0.03
2,2014-01-17,25.670,25.77,25.65,25.710,5615,QYLD,25.700,-0.03,-0.001167,25.820,0.11
3,2014-01-21,25.820,25.74,25.66,25.719,2230,QYLD,25.710,0.11,0.004278,25.529,-0.19
4,2014-01-22,25.529,25.54,25.529,25.530,1055,QYLD,25.719,-0.19,-0.007388,25.530,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
256441,2022-11-29,64.100,64.25,62.71,62.870,2061764,GFS,63.900,0.20,0.003130,62.500,-0.37
256442,2022-11-30,62.500,64.45,60.45,64.350,2418476,GFS,62.870,-0.37,-0.005885,64.320,-0.03
256443,2022-12-01,64.320,64.43,61.87,63.660,1617077,GFS,64.350,-0.03,-0.000466,61.880,-1.78
256444,2022-12-02,61.880,62.69,60.72,62.540,1755186,GFS,63.660,-1.78,-0.027961,62.440,-0.10


In [11]:
test_start = datetime.strptime('2022-01-01', '%Y-%m-%d').date().isoformat()
test_end = datetime.strptime('2022-12-01', '%Y-%m-%d').date().isoformat()

date = {"date": pd.date_range(test_start, test_end)}
dates = pd.DataFrame(data = date)
dates['wday'] = dates['date'].dt.dayofweek
dates = dates.loc[dates['wday'] != 5]
dates = dates.loc[dates['wday'] != 6]
dates = dates.loc[dates['date'] != '2022-11-24']

sum_data_final = pd.DataFrame()

windows = [1, 3, 5, 10, 15, 20, 30, 45, 60, 90]

for roll in windows:
    all_stock_history['rolling_sum'] = (
        all_stock_history.groupby('symbol')['overnight_return']
            .rolling(roll, min_periods = 1).sum().reset_index(0, drop = True))

    for how_many in range(1, 11):

        test_data = pd.DataFrame()

        for date in dates['date']:

            top_ten = (all_stock_history.loc[all_stock_history['date'] == date.date().isoformat()]
                           .sort_values(['rolling_sum'], ascending = False)).head(how_many)

            tmp = pd.DataFrame({'date': date,
                                'investment': top_ten['close'].sum(),
                                'return': top_ten['next_night_return'].sum()},
                              index = [0])

            test_data = pd.concat([test_data, tmp], axis = 0, ignore_index = True)

        test_data['cum_return'] = test_data['return'].cumsum()
        test_data['cum_invest'] = test_data['investment'].cumsum()
        test_data['avg_invest'] = test_data['cum_invest'] / (test_data.index + 1)

        test_data['daily_return_rate'] = test_data['return'] / test_data['investment']
        test_data['cumulative_return_rate'] = test_data['cum_return'] / test_data['avg_invest']

        test_data['month'] = test_data['date'].dt.month
        for mon in range(1, 13):
            sum_data = test_data.loc[test_data['month'] == mon].agg({'investment': ['mean'], 'return': ['sum']})
        
            tmp = pd.DataFrame({
                'num_stocks': how_many,
                'window_size': roll,
                'month': mon,
                'mean_investment': sum_data.iloc[0, 0],
                'total_return': sum_data.iloc[1, 1],
                'return_rate': sum_data.iloc[1, 1] / sum_data.iloc[0, 0],
            }, index = [0])

            sum_data_final = pd.concat([sum_data_final, tmp], axis = 0, ignore_index = True)

sum_data_final

,num_stocks,window_size,month,mean_investment,total_return,return_rate
0,1,1,1,317.387619,-6.215000,-0.019582
1,1,1,2,247.980000,1.840000,0.007420
2,1,1,3,298.777826,-0.880000,-0.002945
3,1,1,4,307.590000,-5.430000,-0.017653
4,1,1,5,234.695909,-10.200000,-0.043460
...,...,...,...,...,...,...
1195,10,90,8,1235.692826,-19.298000,-0.015617
1196,10,90,9,1174.409629,-30.117835,-0.025645
1197,10,90,10,1640.859286,25.820000,0.015736
1198,10,90,11,2441.200476,57.205000,0.023433


In [12]:
(
    sum_data_final
        .loc[sum_data_final['month'] < 12]
        .sort_values(['month', 'return_rate'], ascending = False)
        .groupby('month')
        .head(1)
)

,num_stocks,window_size,month,mean_investment,total_return,return_rate
490,1,15,11,329.822857,40.2050,0.121899
9,1,1,10,304.360000,6.0700,0.019943
128,1,3,9,375.015455,93.6300,0.249670
7,1,1,8,309.080435,25.4400,0.082309
126,1,3,7,204.849048,9.3600,0.045692
125,1,3,6,220.222273,14.5600,0.066115
124,1,3,5,250.859091,9.5850,0.038209
963,1,60,4,415.813333,28.7400,0.069118
722,1,30,3,300.019130,38.2000,0.127325
1,1,1,2,247.980000,1.8400,0.007420


In [18]:
(
    sum_data_final
        .loc[sum_data_final['month'] < 12]
        .groupby(['num_stocks', 'window_size'])
        .agg({'total_return': ['mean'], 
              'return_rate': ['mean']})
).sort_values([('total_return', 'mean')], ascending = False)

total_return return_rate
                               mean        mean
num_stocks window_size                         
1          3              12.459727    0.038174
2          3               8.024727    0.010521
1          5               0.833909   -0.015139
           1              -0.035909   -0.002685
2          1              -0.567855   -0.000683
...                             ...         ...
7          60            -55.412365   -0.041194
10         45            -55.440095   -0.040106
           60            -56.591604   -0.035561
8          60            -58.331228   -0.040360
9          60            -59.770901   -0.039157

[100 rows x 2 columns]

In [29]:
test_start = datetime.strptime('2022-01-01', '%Y-%m-%d').date().isoformat()
test_end = datetime.strptime('2022-12-01', '%Y-%m-%d').date().isoformat()

date = {"date": pd.date_range(test_start, test_end)}
dates = pd.DataFrame(data = date)
dates['wday'] = dates['date'].dt.dayofweek
dates = dates.loc[dates['wday'] != 5]
dates = dates.loc[dates['wday'] != 6]
dates = dates.loc[dates['date'] != '2022-11-24']

sum_data_final = pd.DataFrame()

windows = [1, 3, 5, 10, 15, 20, 30, 45, 60, 90]

for roll in windows:
    var_name = f'rolling_sum_{roll}'
    all_stock_history[var_name] = (all_stock_history.groupby('symbol')['overnight_return']
            .rolling(roll).sum().reset_index(0, drop = True)) 
    
    all_stock_history[var_name] = all_stock_history[var_name] / all_stock_history['close']

all_stock_history    

,date,open,high,low,close,volume,symbol,yest_close,overnight_return,overnight_return_perc,tom_open,next_night_return,rolling_sum,rolling_sum_1,rolling_sum_3,rolling_sum_5,rolling_sum_10,rolling_sum_15,rolling_sum_20,rolling_sum_30,rolling_sum_45,rolling_sum_60,rolling_sum_90,winner
0,2014-01-15,25.750,25.75,25.6394,25.670,6503,QYLD,NaN,NaN,NaN,25.690,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,2014-01-16,25.690,25.7,25.6413,25.700,2910,QYLD,25.670,0.02,0.000779,25.670,-0.03,0.020,0.000778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2014-01-17,25.670,25.77,25.65,25.710,5615,QYLD,25.700,-0.03,-0.001167,25.820,0.11,-0.010,-0.001167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,2014-01-21,25.820,25.74,25.66,25.719,2230,QYLD,25.710,0.11,0.004278,25.529,-0.19,0.100,0.004277,0.003888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2014-01-22,25.529,25.54,25.529,25.530,1055,QYLD,25.719,-0.19,-0.007388,25.530,0.00,-0.090,-0.007442,-0.004309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256441,2022-11-29,64.100,64.25,62.71,62.870,2061764,GFS,63.900,0.20,0.003130,62.500,-0.37,-16.372,0.003181,-0.017178,-0.018610,-0.027676,0.048036,0.071258,0.050819,-0.041467,-0.138969,-0.260410,False
256442,2022-11-30,62.500,64.45,60.45,64.350,2418476,GFS,62.870,-0.37,-0.005885,64.320,-0.03,-16.262,-0.005750,-0.014141,-0.025796,-0.070707,-0.034188,0.042269,0.014685,-0.056830,-0.145874,-0.252712,False
256443,2022-12-01,64.320,64.43,61.87,63.660,1617077,GFS,64.350,-0.03,-0.000466,61.880,-1.78,-16.032,-0.000471,-0.003142,-0.023249,-0.054744,-0.005655,0.041942,0.023249,-0.045350,-0.140072,-0.251838,False
256444,2022-12-02,61.880,62.69,60.72,62.540,1755186,GFS,63.660,-1.78,-0.027961,62.440,-0.10,-18.852,-0.028462,-0.034858,-0.043492,-0.058203,-0.064599,0.047010,-0.011832,-0.055245,-0.164966,-0.301439,False


In [41]:
all_stock_history.loc[all_stock_history['next_night_return'] > 0, 'winner'] = 1
all_stock_history.loc[all_stock_history['next_night_return'] <= 0, 'winner'] = 0
all_stock_history

,date,open,high,low,close,volume,symbol,yest_close,overnight_return,overnight_return_perc,tom_open,next_night_return,rolling_sum,rolling_sum_1,rolling_sum_3,rolling_sum_5,rolling_sum_10,rolling_sum_15,rolling_sum_20,rolling_sum_30,rolling_sum_45,rolling_sum_60,rolling_sum_90,winner
0,2014-01-15,25.750,25.75,25.6394,25.670,6503,QYLD,NaN,NaN,NaN,25.690,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2014-01-16,25.690,25.7,25.6413,25.700,2910,QYLD,25.670,0.02,0.000779,25.670,-0.03,0.020,0.000778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2014-01-17,25.670,25.77,25.65,25.710,5615,QYLD,25.700,-0.03,-0.001167,25.820,0.11,-0.010,-0.001167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2014-01-21,25.820,25.74,25.66,25.719,2230,QYLD,25.710,0.11,0.004278,25.529,-0.19,0.100,0.004277,0.003888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2014-01-22,25.529,25.54,25.529,25.530,1055,QYLD,25.719,-0.19,-0.007388,25.530,0.00,-0.090,-0.007442,-0.004309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256441,2022-11-29,64.100,64.25,62.71,62.870,2061764,GFS,63.900,0.20,0.003130,62.500,-0.37,-16.372,0.003181,-0.017178,-0.018610,-0.027676,0.048036,0.071258,0.050819,-0.041467,-0.138969,-0.260410,0
256442,2022-11-30,62.500,64.45,60.45,64.350,2418476,GFS,62.870,-0.37,-0.005885,64.320,-0.03,-16.262,-0.005750,-0.014141,-0.025796,-0.070707,-0.034188,0.042269,0.014685,-0.056830,-0.145874,-0.252712,0
256443,2022-12-01,64.320,64.43,61.87,63.660,1617077,GFS,64.350,-0.03,-0.000466,61.880,-1.78,-16.032,-0.000471,-0.003142,-0.023249,-0.054744,-0.005655,0.041942,0.023249,-0.045350,-0.140072,-0.251838,0
256444,2022-12-02,61.880,62.69,60.72,62.540,1755186,GFS,63.660,-1.78,-0.027961,62.440,-0.10,-18.852,-0.028462,-0.034858,-0.043492,-0.058203,-0.064599,0.047010,-0.011832,-0.055245,-0.164966,-0.301439,0


In [75]:
model_data = all_stock_history.iloc[90:-1, ]
model_data.replace([np.inf, -np.inf], np.nan, inplace = True)
model_data = model_data.dropna()
model_data.dtypes

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_14378/596616226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.replace([np.inf, -np.inf], np.nan, inplace = True)


date                      object
open                     float64
high                      object
low                       object
close                    float64
volume                     int64
symbol                    object
yest_close               float64
overnight_return         float64
overnight_return_perc    float64
tom_open                 float64
next_night_return        float64
rolling_sum              float64
rolling_sum_1            float64
rolling_sum_3            float64
rolling_sum_5            float64
rolling_sum_10           float64
rolling_sum_15           float64
rolling_sum_20           float64
rolling_sum_30           float64
rolling_sum_45           float64
rolling_sum_60           float64
rolling_sum_90           float64
winner                   float64
dtype: object

In [76]:
target = np.array(model_data['winner'])

features = model_data[['volume', 'overnight_return_perc', 'rolling_sum_1',
                       'rolling_sum_3', 'rolling_sum_5', 'rolling_sum_10',
                       'rolling_sum_15', 'rolling_sum_20', 'rolling_sum_30',
                       'rolling_sum_60', 'rolling_sum_90']]
features['overnight_return_perc'] = features['overnight_return_perc'].astype('float32')
features['rolling_sum_1'] = features['rolling_sum_1'].astype('float32')
features['rolling_sum_3'] = features['rolling_sum_3'].astype('float32')
features['rolling_sum_5'] = features['rolling_sum_5'].astype('float32')
features['rolling_sum_10'] = features['rolling_sum_10'].astype('float32')
features['rolling_sum_15'] = features['rolling_sum_15'].astype('float32')
features['rolling_sum_20'] = features['rolling_sum_20'].astype('float32')
features['rolling_sum_30'] = features['rolling_sum_30'].astype('float32')
features['rolling_sum_60'] = features['rolling_sum_60'].astype('float32')
features['rolling_sum_90'] = features['rolling_sum_90'].astype('float32')

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_14378/1785765785.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['overnight_return_perc'] = features['overnight_return_perc'].astype('float32')
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_14378/1785765785.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['rolling_sum_1'] = features['rolling_sum_1'].astype('float32')
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_14378/1785765785.py:9: SettingWithCop

In [77]:
train_features, test_features, train_labels, test_labels = train_test_split(features, 
                                                                            target, 
                                                                            test_size = 0.25, 
                                                                            random_state = 31786)

In [78]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (187516, 11)
Training Labels Shape: (187516,)
Testing Features Shape: (62506, 11)
Testing Labels Shape: (62506,)


In [79]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf.fit(train_features, train_labels)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [80]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


Mean Absolute Error: 0.47 degrees.


In [84]:
confusion_matrix(test_labels, predictions)


array([[13869, 16476],
       [13054, 19107]])

In [85]:
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: volume               Importance: 0.1
Variable: rolling_sum_3        Importance: 0.09
Variable: rolling_sum_5        Importance: 0.09
Variable: rolling_sum_10       Importance: 0.09
Variable: rolling_sum_15       Importance: 0.09
Variable: rolling_sum_20       Importance: 0.09
Variable: rolling_sum_30       Importance: 0.09
Variable: rolling_sum_60       Importance: 0.09
Variable: rolling_sum_90       Importance: 0.09
Variable: overnight_return_perc Importance: 0.08
Variable: rolling_sum_1        Importance: 0.08


In [95]:
fpr, tpr, threshold = roc_curve(test_labels, predictions)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
lw = 2
plt.plot(
    fpr[2],
    tpr[2],
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % roc_auc[2],
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()